In [1]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
ppp = os.path.abspath('../../visualization')

sys.path.insert(1, ppp)
sys.path.insert(1, p)
sys.path.insert(1, pp)




In [2]:
from gc import collect

from distribution_collector import collect_tag_distributions
from utils.statistics import calculate_gini, calculate_entropy, get_dist_x_div, calc_regression
from distribution_collector import (collect_topic_distributions)
from constants import CONSTANTS

from matplotlib import pyplot as plt
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
import lib.stats.stats as st


ModuleNotFoundError: No module named 'distribution_collector'

In [ ]:
tag_info = "../../visualization/result/tag/tag_info.json"
tag_dir = f'../../visualization/{CONSTANTS.tag_difficulty_data_dir[3:]}'
lda_dir = f'../../visualization/{CONSTANTS.lda_difficulty_data_dir[3:]}'
diff_dict = {'0' : 'Basic', '1' : 'Intermediate', '2' : 'Advanced'}
output_dir = './fig/'

In [ ]:
print(tag_dir)

In [ ]:
bert_topic_distribution = dict()
for diff_str in diff_dict.keys():
    bert_topic_distribution[diff_str] = collect_topic_distributions(window=7,
                                                                data_dir=tag_dir,
                                                                options={'difficulties': [diff_str]})
    

In [ ]:
tag_distribution = dict()
for diff_str in diff_dict.keys():
    print(diff_str)
    tag_distribution[diff_str] = collect_tag_distributions(window=7,
                                                          tag_info = tag_info,
                                                          data_dir=tag_dir,
                                                          options={'difficulties': [diff_str]})
    

In [ ]:
b_gini_dict, b_entropy_dict = dict(), dict()
for diff_str, topic_distribution in bert_topic_distribution.items():
    b_entropy_dict[diff_str] = list(map(lambda x: calculate_entropy(list(x.values())), topic_distribution))
    b_gini_dict[diff_str] = list(map(lambda x: calculate_gini(list(x.values())), topic_distribution))
    


In [ ]:
t_gini_dict, t_entropy_dict = dict(), dict()
for diff_str, distribution in tag_distribution.items():
    t_entropy_dict[diff_str] = list(map(lambda x: calculate_entropy(list(x.values())), distribution))
    t_gini_dict[diff_str] = list(map(lambda x: calculate_gini(list(x.values())), distribution))
    


In [ ]:
b_coef_dict = dict()
b_coef_dict[0] = b_gini_dict
b_coef_dict[1] = b_entropy_dict

In [ ]:
alpha_dict = {0 : 'A. ', 1  : 'B. ', 2 : 'C. '}

In [ ]:
t_coef_dict = dict()
t_coef_dict[0] = t_gini_dict
t_coef_dict[1] = t_entropy_dict

In [ ]:
def calc_chow(x, y):
    st_ = st.Stats(np.array(x), np.array(y), 2, 0.95)
    F_stat, p_value = st_.chow_test()
    return st_, F_stat, p_value
    

In [ ]:
def draw_scatter_line_plot(dict_):
    
    v_size = len(dict_[0].keys())
    calc_type_dict = { 0 : 'Gini Coefficient', 1 : 'Entropy'}

    fig, axs = plt.subplots(nrows=2, ncols=v_size,  figsize = (18, 12), sharey='row', sharex=True)
    coef_dict = dict_[0]
    for v_idx in coef_dict.keys():
        list_ = coef_dict[v_idx]
        x_rel, divider = get_dist_x_div(list_)
        st_, F_stat_, p_value_ = calc_chow(x_rel, list_)


        h_idx, v_idx = 0, int(v_idx)
        axs[0][v_idx].scatter(x_rel, list_, color = 'darkgray', alpha = 0.7,  s=10, marker='x')
        axs[0][v_idx].plot(x_rel, st_.y_predict, label="Overall Fit", linestyle="--", color="black")  

        axs[0][v_idx].plot(x_rel[:divider], st_.y1_predict, label="before ChatGPT", linewidth='2')
        axs[0][v_idx].fill_between(x_rel[:divider], st_.y1_predict - st_.y1_conf_interval, st_.y1_predict + st_.y1_conf_interval, alpha=0.1)

        axs[0][v_idx].plot(x_rel[divider:], st_.y2_predict, label="after ChatGPT", linewidth='2')
        axs[0][v_idx].fill_between(x_rel[divider:], st_.y2_predict - st_.y2_conf_interval, st_.y2_predict + st_.y2_conf_interval, alpha=0.1)

        axs[0][v_idx].axvline(x=0, color='tab:red', linestyle='-.', linewidth=1)
        # axs[0][v_idx].set_xlabel("Week Relative to ChatGPT Release", fontsize = 10)
        axs[0][v_idx].set_ylabel(f"{calc_type_dict[h_idx]}", fontsize = 22)
        # axs[0][v_idx].text( 0.05, 0.05, transform=axs[0][v_idx].transAxes, s=f'F-Statistic : {F_stat_:.2f}\np-value : {p_value_:.4f}', fontsize = 10) 
        axs[0][v_idx].set_title(f"{alpha_dict[v_idx]}{diff_dict[str(v_idx)]} Level")
        axs[0][v_idx].grid(True, linestyle='--', alpha=0.3)
        
    

    coef_dict = dict_[1]
    for v_idx in coef_dict.keys():
        list_ = coef_dict[v_idx]
        x_rel, divider = get_dist_x_div(list_)
        st_, F_stat_, p_value_ = calc_chow(x_rel, list_)

        h_idx, v_idx = 1, int(v_idx)
        axs[1][v_idx].scatter(x_rel, list_, color = 'darkgray', alpha = 0.7,  s=10, marker='x')
        axs[1][v_idx].plot(x_rel, st_.y_predict, label="Full Model", linestyle="--", color="black")  

        axs[1][v_idx].plot(x_rel[:divider], st_.y1_predict, label="before chatGPT", linewidth='2')
        axs[1][v_idx].fill_between(x_rel[:divider], st_.y1_predict - st_.y1_conf_interval, st_.y1_predict + st_.y1_conf_interval, alpha=0.1)

        axs[1][v_idx].plot(x_rel[divider:], st_.y2_predict, label="after chatGPT", linewidth='2')
        axs[1][v_idx].fill_between(x_rel[divider:], st_.y2_predict - st_.y2_conf_interval, st_.y2_predict + st_.y2_conf_interval, alpha=0.1)

        axs[1][v_idx].axvline(x=0, color='tab:red', linestyle='-.', linewidth=1)
        axs[1][v_idx].set_xlabel("Week Relative to ChatGPT Release", fontsize = 10)
        axs[1][v_idx].set_ylabel(f"{calc_type_dict[h_idx]}", fontsize = 10)
        axs[1][v_idx].set_title(f"{alpha_dict[v_idx]}{diff_dict[str(v_idx)]} Level")
        axs[1][v_idx].text( 0.05, 0.05, transform=axs[1][v_idx].transAxes, s=f'F-Statistic : {F_stat_:.2f}\np-value : {p_value_:.4f}', fontsize = 10) 
        axs[1][v_idx].grid(True, linestyle='--', alpha=0.3)
        
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.savefig(f"{output_dir}Result_Fig6.png", dpi=300, bbox_inches='tight')


In [ ]:
draw_scatter_line_plot(b_coef_dict)

In [ ]:
draw_scatter_line_plot(t_coef_dict)

In [ ]:

fig, axs = plt.subplots(1, 3, figsize = (18, 6), sharey=True, sharex=True)

coef_dict = t_coef_dict[1]
for v_idx in coef_dict.keys():
    list_ = coef_dict[v_idx]
    x_rel, divider = get_dist_x_div(list_)
    st_, F_stat_, p_value_ = calc_chow(x_rel, list_)

    h_idx, v_idx = 1, int(v_idx)
    axs[v_idx].scatter(x_rel, list_, color = 'darkgray', alpha = 0.7,  s=10, marker='x')
    axs[v_idx].plot(x_rel, st_.y_predict, label="Full Model", linestyle="--", color="black")  

    axs[v_idx].plot(x_rel[:divider], st_.y1_predict, label="before chatGPT", linewidth='2')
    axs[v_idx].fill_between(x_rel[:divider], st_.y1_predict - st_.y1_conf_interval, st_.y1_predict + st_.y1_conf_interval, alpha=0.1)

    axs[v_idx].plot(x_rel[divider:], st_.y2_predict, label="after chatGPT", linewidth='2')
    axs[v_idx].fill_between(x_rel[divider:], st_.y2_predict - st_.y2_conf_interval, st_.y2_predict + st_.y2_conf_interval, alpha=0.1)

    axs[v_idx].axvline(x=0, color='tab:red', linestyle='-.', linewidth=1)
    axs[v_idx].set_xlabel("Week Relative to ChatGPT Release", fontsize = 15)
    axs[v_idx].set_ylabel("Entropy", fontsize = 15)
    axs[v_idx].set_title(f"{diff_dict[str(v_idx)]} Level", fontsize = 20)
    axs[v_idx].text( 0.05, 0.05, transform=axs[v_idx].transAxes, s=f'F-Statistic : {F_stat_:.2f}\np-value : {p_value_:.4f}', fontsize = 10) 
    axs[v_idx].grid(True, linestyle='--', alpha=0.3)
    axs[v_idx].legend( loc='lower right')

fig.tight_layout()
plt.savefig(f"{output_dir}Result_Fig6.png", dpi=300, bbox_inches='tight')
